In [6]:
import os

import matplotlib

import numpy as np
import pandas as pd

from scipy import stats

from matplotlib import pyplot as plt

In [7]:
# Rutas de archivos.
path_i = "../data/Indices/"
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)

dates = pd.read_csv(path_d + "onset_CHIRPS.csv", index_col = "Año")
# Se convierten las fechas a días julianos.
for i in [0, 1]:
    dates.iloc[:, i] = dates.iloc[:, i].apply(
        lambda x: pd.to_datetime(x) ).apply(lambda x: x.dayofyear)
dates.iloc[:, 2] = dates.iloc[:, 2].apply(lambda x: x[:-5]).astype(int)

cols = ["Año"] + [i for i in range(1, 13)]

BEST_1 = pd.read_table( path_i + "BEST_1M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]
BEST_3 = pd.read_table( path_i + "BEST_3M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]
BEST_5 = pd.read_table( path_i + "BEST_5M", sep = "\s+",
    names = cols, index_col = 0 ).loc[1981:2020]

BEST_1_Y = BEST_1.mean(axis = 1)
BEST_3_Y = BEST_3.mean(axis = 1)
BEST_5_Y = BEST_5.mean(axis = 1)

BEST = [BEST_1, BEST_3, BEST_5]
BEST_Y = [BEST_1_Y, BEST_3_Y, BEST_5_Y]

# Periodo de media móvil a usar.
m = 0

In [11]:
# Onset

c = 0

# Confianza
a = 0.05

for i in range(c, c + 1):

    print("\n" + dates.columns[i])

    print("\nYearly correlations:")
    for k in range(0, 3):
        corr = stats.pearsonr( BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        print( f"Year {-k:-2}: " + f"{corr}" )

    for k in range(0, 3):
        print(f"\nMonthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr = stats.pearsonr( BEST[m].shift(k).iloc[k:, j],
                dates.iloc[k:, i] )
            print( f"{ j + 1 :02d}: " + f"{corr}" )


Cuenca - Onset

Yearly correlations:
Year  0: (-0.32967269101817465, 0.03776561094621329)
Year -1: (0.04765818559059591, 0.7732659052223905)
Year -2: (0.020927903417862815, 0.9007513624853272)

Monthly correlations, year  0:
01: (-0.14904903684946072, 0.3586614740325026)
02: (0.021845834250376218, 0.8935608759942482)
03: (-0.11715662979504313, 0.4715541130431742)
04: (-0.17283966354383157, 0.2861872661060657)
05: (-0.2335280833638647, 0.14698424048290623)
06: (-0.22383811378529117, 0.164992142397954)
07: (-0.35461825233817923, 0.024754770765920826)
08: (-0.2862718266210387, 0.07332240153307952)
09: (-0.36427244454403895, 0.020841897365844332)
10: (-0.38580844813958787, 0.013943394105855248)
11: (-0.32127304780009447, 0.0432364790877349)
12: (-0.30587725458426757, 0.054923477212715795)

Monthly correlations, year -1:
01: (0.16254838437785457, 0.32281843673768085)
02: (0.1277945312227517, 0.43816376848458444)
03: (0.10239279823673851, 0.5350787597324019)
04: (0.21923882218148838, 0.1799

In [42]:
# Correlation boostrapping.

def pcorr(x, y, axis):
    if x.ndim == 1: 
        return stats.pearsonr(x, y)[0]
    else:
        n = y.shape[0]
        statistic = np.empty( (n,) )
        for i in range(n): statistic[i], _ = stats.pearsonr(x[i, :], y[i, :])
        return statistic

In [44]:
# Onset.

c = 0

# Confianza
a = 0.05

for i in range(c, c + 1):

    print(dates.columns[i])

    print("Yearly correlations:")
    for k in range(0, 3):
        corr, _ = stats.pearsonr( 
            BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
        int = stats.bootstrap( 
            [ BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] ],
            pcorr, confidence_level = 0.95, method = "basic" 
            ).confidence_interval
        print( f"Year {-k:-2}: " + f"{int}" )

    for k in range(0, 2):
        print(f"Monthly correlations, year {-k:-2}:")
        for j in range(0, 12):
            corr, _ = stats.pearsonr(
                BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] )
            int = stats.bootstrap( 
                [ BEST[m].shift(k).iloc[k:, j], dates.iloc[k:, i] ],
                pcorr, confidence_level = 0.95, method = "basic" 
                ).confidence_interval
            print( f"{ j + 1 :02d}: " + f"{corr:-8.5f}" )

Cuenca - Onset
Yearly correlations:
Year  0: ConfidenceInterval(low=-0.9643846955311306, high=-0.34175260046762823)
Year -1: ConfidenceInterval(low=-0.24474907442903104, high=0.3872254538830992)
Year -2: ConfidenceInterval(low=-0.2636471374893489, high=0.37393874854610076)
Monthly correlations, year  0:
01: -0.05643
02: -0.08015
03: -0.08081
04: -0.18946
05: -0.23189
06: -0.28720
07: -0.30122
08: -0.34221
09: -0.35485
10: -0.36627
11: -0.34534
12: -0.34295
Monthly correlations, year -1:
01:  0.11689
02:  0.13854
03:  0.15646
04:  0.16241
05:  0.15070
06:  0.04432
07: -0.00905
08: -0.07701
09: -0.08294
10: -0.08853
11: -0.05302
12: -0.08906


In [69]:
# Onset

c = 0
m = 1

# Confianza
a = 0.05

print("\nYearly correlations:")
for k in range(0, 3):
    corr = stats.pearsonr( BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
    print( f"Year {-k:-2}: " + f"{corr}" )

for k in range(0, 3):
    corr, _ = stats.pearsonr( 
        BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] )
    int = stats.bootstrap( 
        [ BEST_Y[m].shift(k).iloc[k:], dates.iloc[k:, i] ],
        pcorr, confidence_level = (0.174), method = "percentile" 
        ).confidence_interval
    print( f"Year {-k:-2}: " + f"{int}" )


Yearly correlations:
Year  0: (-0.32769199609323457, 0.03900180803500559)
Year -1: (0.03647207524198056, 0.8255357271271604)
Year -2: (0.027017483243571772, 0.8720834199253136)
Year  0: ConfidenceInterval(low=-0.0380894354580636, high=0.03416150968449266)
Year -1: ConfidenceInterval(low=-0.035717173782240534, high=0.03795353130678643)
Year -2: ConfidenceInterval(low=-0.036544704414758024, high=0.0345681698110351)
